In [26]:
# Script to give stats about a crawl (quickly)
# Author Hadi, 25 May 2018


import pandas as pd
#import json
#import re
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
#import matplotlib
from collections import Counter
from urllib.parse import urlparse


# TODO: turn this into a script (with parts bellow)
# TODO: below is good. we can scale up, I think. just to figure out that one error. if important.


In [27]:
#INPUT_DIR = "/home/hadi/Devel/CookieLaws18/out/crawl-vpnDE-20180524.2119-1/"
INPUT_DIR = "/home/hadi/Devel/CookieLaws18/out/crawl-vpnUS-20180524.2124-1/"

In [28]:
import sqlite3
dbcon = sqlite3.connect(INPUT_DIR + "crawl-data.sqlite")

# sqlite reference: https://sqlite.org/lang_corefunc.html
sites = pd.read_sql_query("""SELECT s.visit_id, 
                            site_url as site, 
                            bool_success, 
                            group_concat(response_status) as response_status, 
                            group_concat(location) as redirect, 
                            count(*) as response_count
                          FROM site_visits s 
                          LEFT JOIN CrawlHistory h
                               ON s.site_url=h.arguments AND h.command='GET'
                          LEFT JOIN http_responses r
                               ON s.visit_id=r.visit_id AND LOWER(RTRIM(s.site_url))=RTRIM(r.url, '/')
                          GROUP BY s.visit_id""", 
                          dbcon)

print("SUCCESS: ", sites.groupby('bool_success').count().site)

# HTTP CODES.
t = sites[(sites.response_count==1)&(sites.bool_success==1)].copy()
print("HTTP CODE", t.groupby("response_status").count().site)

SUCCESS:  bool_success
-1     16
 0     34
 1    508
Name: site, dtype: int64
HTTP CODE response_status
200     52
301    356
302     84
303      1
307      3
403      4
404      3
503      2
Name: site, dtype: int64


In [29]:
# COOKIES. 
cookies = pd.read_sql_query("""SELECT site_url as site, host as chost, name, value, 
                                      is_session, change, expiry, creationTime
                            FROM site_visits s 
                            LEFT JOIN javascript_cookies c 
                            ON s.visit_id=c.visit_id ORDER BY s.visit_id""", 
                            dbcon)


print("TOTAL COOKIES", cookies.count().is_session)
print("AVERAGE COOKIES", np.mean(cookies.groupby("site").count().is_session))

TOTAL COOKIES 25857
AVERAGE COOKIES 46.33870967741935


In [31]:
# JS CALLS
js_calls = pd.read_sql_query("""SELECT site_url as site, script_url as js_url, script_url as jhost
                                FROM site_visits s LEFT JOIN javascript j
                                ON s.visit_id=j.visit_id 
                                WHERE script_url is not null
                                ORDER BY s.visit_id""", dbcon)

print("TOTAL JS CALLS", js_calls.count().js_url)
print("AVERAGE JS CALLS", np.mean(js_calls.groupby("site").count().js_url))

TOTAL JS CALLS 230293
AVERAGE JS CALLS 477.7863070539419


In [42]:
# NUMBER SCREENSHOTS; NUMBER HTML PAGES
from glob import glob
print("screenshots: ", len(glob(INPUT_DIR + "screenshots/*")))
print("page sources: ", len(glob(INPUT_DIR + "sources/*")))

# TODO: could also get sizes

screenshots:  0
page sources:  0


In [ ]:
# IF NEEDED TO GET NUM TLDS IN VISITED
# (Helper function to remove the http, https and extract domains from site_url)

from tldextract import extract as tldextract  # pip install tldextract

def strip_site(site):
    """Removes leading http:// or https:// and trailing '/'"""
    if site is None:
        return None
    site = site.lower().strip()
    if site.startswith('http'):
        site = site.replace('https://', '').replace('http://', '')
    if site.startswith('www.'):
        site = site[4:]
    if site.endswith('/'):
        site = site[:-1]
    return site

def extract_domain(site):
    """Returns domain+tld from a full domain"""
    site = strip_site(site)  # just to be sure
    ext = tldextract(site)
    ret = ".".join([ss for ss in ext[-2:] if ss])
    # In case of gov.ie, or IPs, our 'ret' works better than 'ext.registered_domain'
    return ret